In [2]:
import pandas as pd
import requests as r
from io import StringIO

In [87]:
def get_page(year):
    url = f'https://en.wikipedia.org/wiki/List_of_American_films_of_{year}'
    page_load = r.get(url)
    string = StringIO(page_load.text)
    dfs = pd.read_html(string,extract_links="all")
    sub_dfs = []
    for this_table in dfs:
        if ('Opening', None) in this_table.columns:
            sub_dfs.append(this_table)
    concatted = pd.concat(sub_dfs)
    concatted['year'] = year
    return(concatted)


In [88]:
all_years = {}
for year in range(1970,2024):
    if year in all_years:
        print(f'Skipping {year}')
        break
    print(f'Getting {year}')
    this_year = get_page(year)
    all_years[year] = this_year

Getting 1970
Getting 1971
Getting 1972
Getting 1973
Getting 1974
Getting 1975
Getting 1976
Getting 1977
Getting 1978
Getting 1979
Getting 1980
Getting 1981
Getting 1982
Getting 1983
Getting 1984
Getting 1985
Getting 1986
Getting 1987
Getting 1988
Getting 1989
Getting 1990
Getting 1991
Getting 1992
Getting 1993
Getting 1994
Getting 1995
Getting 1996
Getting 1997
Getting 1998
Getting 1999
Getting 2000
Getting 2001
Getting 2002
Getting 2003
Getting 2004
Getting 2005
Getting 2006
Getting 2007
Getting 2008
Getting 2009
Getting 2010
Getting 2011
Getting 2012
Getting 2013
Getting 2014
Getting 2015
Getting 2016
Getting 2017
Getting 2018
Getting 2019
Getting 2020
Getting 2021
Getting 2022
Getting 2023


In [89]:
all_years_pd = pd.concat([all_years[k] for k in all_years])

In [131]:
col_names = ['month','day','title','production_company','cast_and_crew','ref','production_company_2']
# col_dict = {all_years_pd.columns[i]:col_names[i] for i in range(len(col_names))}
renamed = all_years_pd.copy()
for i in range(len(col_names)):
    old_name = all_years_pd.columns[i]
    print(f'Creating column {col_names[i]} from {old_name}')
    renamed[col_names[i]] = all_years_pd[old_name]
    print(f'\tDropping column {old_name}')
    renamed = renamed.drop(old_name, axis=1)

Creating column month from ('Opening', None)
	Dropping column ('Opening', None)
Creating column day from ('Opening', None).1
	Dropping column ('Opening', None).1
Creating column title from ('Title', None)
	Dropping column ('Title', None)
Creating column production_company from ('Production company', None)
	Dropping column ('Production company', None)
Creating column cast_and_crew from ('Cast and crew', None)
	Dropping column ('Cast and crew', None)
Creating column ref from ('Ref.', None)
	Dropping column ('Ref.', None)
Creating column production_company_2 from year
	Dropping column year


In [150]:
def extract_first(val):
    try:
        return val[0]
    except:
        return val
def extract_second(val):
    try:
        return val[1]
    except:
        return val

In [162]:
cleaned = renamed.copy()
cleaned['month'] = cleaned['month'].apply(extract_first)
cleaned['day'] = cleaned['day'].apply(extract_first)
cleaned['title_text'] = cleaned['title'].apply(extract_first)
cleaned['title_href'] = cleaned['title'].apply(extract_second)
cleaned['production_company_text'] = cleaned['production_company'].apply(extract_first)
cleaned['production_company_href'] = cleaned['production_company'].apply(extract_second)
cleaned = cleaned.drop([('', None),("Production Company",None),"production_company_2"],axis=1)

In [165]:
cleaned.to_csv('data/raw/movies_list_with_links.csv')